<a href="https://colab.research.google.com/github/sushirito/Methylmercury/blob/main/TST_Boltzmann.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## This code was inspired by: https://github.com/lbm-principles-practice/code/blob/master/chapter9/shanchen.cpp



In [1]:
import numpy as np
import matplotlib.pyplot as plt
import os
from matplotlib.animation import FuncAnimation
from IPython.display import HTML
from tabulate import tabulate

# Lattice parameters
nx = 64
ny = 64
nsteps = 500
noutput = 100
nfluids = 1
tau = 1

# Shan-Chen parameters for a single fluid
gA = -0.16
rho0 = 1.0
rhol = 2.1
rhog = 0.15

# Solid-fluid interaction parameters
W = 0.15

# Gravitational force
g_gravity = 9.81e-5

# Fixed parameters
npop = 9
cx = np.array([0, 1, 0, -1, 0, 1, -1, -1, 1])
cy = np.array([0, 0, 1, 0, -1, 1, 1, -1, -1])
weight = np.array([4./9., 1./9., 1./9., 1./9., 1./9., 1./36., 1./36., 1./36., 1./36.])

# Weights for solid-fluid interactions
t_w = np.array([0, 2, 2, 2, 2, 1, 1, 1, 1])

# Arrays
rho = np.zeros(nx * ny)
press = np.zeros(nx * ny)
ux = np.zeros(nx * ny)
uy = np.zeros(nx * ny)
Fx = np.zeros(nx * ny)
Fy = np.zeros(nx * ny)
feq = np.zeros(npop)
forcing = np.zeros(npop)
f1 = np.zeros((nx * ny, npop))
f2 = np.zeros((nx * ny, npop))

def create_elementary_pattern(size=16):
    pattern = np.zeros((size, size), dtype=int)
    center = size // 2
    radius = size // 4
    for y in range(size):
        for x in range(size):
            if (x - center) ** 2 + (y - center) ** 2 <= radius ** 2:
                pattern[y, x] = 1
    return pattern

def create_porous_medium(n_patterns=4):
    elementary_size = 16
    pattern = create_elementary_pattern(elementary_size)
    porous_medium = np.tile(pattern, (n_patterns, n_patterns))
    return porous_medium

solid = create_porous_medium()

def compute_density(pop):
    rho[:] = np.sum(pop, axis=1)

def psi(dens):
    return rho0 * (1.0 - np.exp(-dens / rho0))

def compute_sc_forces():
    for y in range(ny):
        for x in range(nx):
            k = y * nx + x
            Fx[k] = 0.0
            Fy[k] = 0.0

            # Fluid-fluid interactions (reduced to a single fluid)
            fxtemp = 0.0
            fytemp = 0.0
            for i in range(1, npop):
                x2 = (x + cx[i] + nx) % nx
                y2 = (y + cy[i] + ny) % ny
                psinb = psi(rho[y2 * nx + x2])
                fxtemp += t_w[i] * cx[i] * psinb
                fytemp += t_w[i] * cy[i] * psinb
            psiloc = psi(rho[k])
            Fx[k] += -gA * psiloc * fxtemp
            Fy[k] += -gA * psiloc * fytemp

            # Solid-fluid interactions
            fxtemp_solid = 0.0
            fytemp_solid = 0.0
            for i in range(1, npop):
                x2 = (x + cx[i] + nx) % nx
                y2 = (y + cy[i] + ny) % ny
                s_value = solid[y2, x2]
                fxtemp_solid += t_w[i] * cx[i] * s_value
                fytemp_solid += t_w[i] * cy[i] * s_value
            Fx[k] += -W * psiloc * fxtemp_solid
            Fy[k] += -W * psiloc * fytemp_solid

            # Gravity force
            Fy[k] += -rho[k] * g_gravity

    return Fx, Fy

def compute_velocity(pop):
    for y in range(ny):
        for x in range(nx):
            k = y * nx + x
            dens = rho[k]
            if dens > 1e-10 and solid.flatten()[k] == 0:  # Only compute for fluid nodes with non-zero density
                ux[k] = ((pop[k, 1] - pop[k, 3] + pop[k, 5] - pop[k, 6] - pop[k, 7] + pop[k, 8]) + 0.5 * Fx[k]) / dens
                uy[k] = ((pop[k, 2] - pop[k, 4] + pop[k, 5] + pop[k, 6] - pop[k, 7] - pop[k, 8]) + 0.5 * Fy[k]) / dens
            else:
                ux[k] = 0
                uy[k] = 0

def equilibrium(k):
    dens = rho[k]
    vx = ux[k]
    vy = uy[k]
    usq = vx * vx + vy * vy
    feq[0] = weight[0] * dens * (1.0 - 1.5 * usq)
    feq[1] = weight[1] * dens * (1.0 + 3.0 * vx + 4.5 * vx * vx - 1.5 * usq)
    feq[2] = weight[2] * dens * (1.0 + 3.0 * vy + 4.5 * vy * vy - 1.5 * usq)
    feq[3] = weight[3] * dens * (1.0 - 3.0 * vx + 4.5 * vx * vx - 1.5 * usq)
    feq[4] = weight[4] * dens * (1.0 - 3.0 * vy + 4.5 * vy * vy - 1.5 * usq)
    feq[5] = weight[5] * dens * (1.0 + 3.0 * (vx + vy) + 4.5 * (vx + vy) * (vx + vy) - 1.5 * usq)
    feq[6] = weight[6] * dens * (1.0 + 3.0 * (-vx + vy) + 4.5 * (-vx + vy) * (-vx + vy) - 1.5 * usq)
    feq[7] = weight[7] * dens * (1.0 + 3.0 * (-vx - vy) + 4.5 * (vx + vy) * (vx + vy) - 1.5 * usq)
    feq[8] = weight[8] * dens * (1.0 + 3.0 * (vx - vy) + 4.5 * (vx - vy) * (vx - vy) - 1.5 * usq)

def initialisation():
    global f1, f2, rho
    rho = np.zeros(nx * ny)
    rho_i = np.random.uniform(0.5, 1.9)
    for k in range(nx * ny):
        if solid.flatten()[k] == 0:
            rho[k] = max(rho_i, 0.1)  # Ensure minimum density
        else:
            rho[k] = 0

    for k in range(nx * ny):
        ux[k] = uy[k] = 0.0
        equilibrium(k)
        f1[k] = feq
        f2[k] = feq

def bounce_back_boundaries(f):
    for i in range(1, npop):
        f[solid.flatten() == 1, i] = f[solid.flatten() == 1, npop - i]
    return f

def push():
    global f1, f2
    omega = 1.0 / tau
    for y in range(ny):
        for x in range(nx):
            k = y * nx + x
            for i in range(npop):
                forcing[i] = weight[i] * (1.0 - 0.5 * omega) * (
                    (3.0 * (cx[i] - ux[k]) + 9.0 * cx[i] * (cx[i] * ux[k] + cy[i] * uy[k])) * Fx[k] +
                    (3.0 * (cy[i] - uy[k]) + 9.0 * cy[i] * (cx[i] * ux[k] + cy[i] * uy[k])) * Fy[k]
                )
            equilibrium(k)
            for i in range(npop):
                x2 = (x + cx[i] + nx) % nx
                y2 = (y + cy[i] + ny) % ny
                f2[y2 * nx + x2, i] = f1[k, i] * (1.0 - omega) + feq[i] * omega + forcing[i]
    f2 = bounce_back_boundaries(f2)
    f1, f2 = f2, f1

def calculate_saturation():
    rho_threshold = (np.max(rho) + np.min(rho)) / 2
    liquid_nodes = np.sum((rho > rho_threshold) & (solid.flatten() == 0))
    total_fluid_nodes = np.sum(solid.flatten() == 0)
    saturation = liquid_nodes / total_fluid_nodes
    return saturation

output_dir = '/content/lbm_output'
os.makedirs(output_dir, exist_ok=True)

def save_plot(fig, filename):
    filepath = os.path.join(output_dir, filename)
    fig.savefig(filepath)
    plt.close(fig)
    print(f"Saved: {filepath}")

def density_plot(rho, step):
    fig, ax = plt.subplots(figsize=(10, 10))
    im = ax.imshow(rho.reshape(ny, nx), cmap='viridis')
    plt.colorbar(im, ax=ax, label='Density')
    ax.set_title(f'Density Distribution at Step {step}')
    ax.set_xlabel('X')
    ax.set_ylabel('Y')
    save_plot(fig, f'density_plot_{step:05d}.png')

def velocity_magnitude_plot(ux, uy, step):
    velocity_magnitude = np.sqrt(ux**2 + uy**2).reshape(ny, nx)
    fig, ax = plt.subplots(figsize=(10, 10))
    im = ax.imshow(velocity_magnitude, cmap='plasma')
    plt.colorbar(im, ax=ax, label='Velocity Magnitude')
    ax.set_title(f'Velocity Magnitude at Step {step}')
    ax.set_xlabel('X')
    ax.set_ylabel('Y')
    save_plot(fig, f'velocity_magnitude_{step:05d}.png')

def velocity_streamplot(ux, uy, step):
    x = np.arange(0, nx)
    y = np.arange(0, ny)
    X, Y = np.meshgrid(x, y)

    fig, ax = plt.subplots(figsize=(12, 10))
    strm = ax.streamplot(X, Y, ux.reshape(ny, nx), uy.reshape(ny, nx), density=1.5, color='k', linewidth=0.5)
    plt.colorbar(strm.lines, ax=ax, label='Velocity Magnitude')
    ax.set_title(f'Velocity Streamlines at Step {step}')
    ax.set_xlabel('X')
    ax.set_ylabel('Y')
    save_plot(fig, f'velocity_streamlines_{step:05d}.png')

def density_animation(rho_history):
    fig, ax = plt.subplots(figsize=(10, 10))
    ims = []
    for i in range(len(rho_history)):
        im = ax.imshow(rho_history[i].reshape(ny, nx), animated=True, cmap='viridis')
        if i == 0:
            ax.imshow(rho_history[i].reshape(ny, nx), cmap='viridis')
        ims.append([im])

    ani = FuncAnimation(fig, lambda x: ims[x], frames=len(rho_history), interval=50, blit=True, repeat_delay=1000)
    plt.close(fig)

    # Save the animation as HTML
    animation_path = os.path.join(output_dir, 'density_animation.html')
    with open(animation_path, 'w') as f:
        f.write(ani.to_jshtml())
    print(f"Saved animation: {animation_path}")

    return HTML(ani.to_jshtml())

def main():
    initialisation()
    rho_history = []
    saturation_history = []
    phase_separation_table = []

    for step in range(nsteps + 1):
        compute_density(f1)
        compute_sc_forces()
        compute_velocity(f1)
        push()

        if step % noutput == 0:
            print(f"Running time step {step}")
            rho_history.append(rho.copy())
            density_plot(rho, step)
            velocity_magnitude_plot(ux, uy, step)
            velocity_streamplot(ux, uy, step)

            saturation = calculate_saturation()
            rho_history.append(rho.copy())
            saturation_history.append(saturation)

            rho_i = np.mean(rho)
            rho_liq = np.max(rho)
            rho_gas = np.min(rho)
            s_th = (rho_i - rho_gas) / (rho_liq - rho_gas)
            s_cal = saturation

            phase_separation_table.append([rho_i, s_th, rho_liq, rho_gas, s_cal])

    # Generate and display density animation
    animation = density_animation(rho_history)
    display(animation)

    print("\nPhase Separation Table:")
    headers = ["ρi", "S_th", "ρ_liq^cal", "ρ_gas^cal", "S_cal"]
    print(tabulate(phase_separation_table, headers=headers, floatfmt=".3f"))

    plt.figure(figsize=(10, 6))
    plt.plot(range(0, nsteps+1, noutput), saturation_history)
    plt.xlabel('Time Step')
    plt.ylabel('Saturation')
    plt.title('Saturation Evolution')
    plt.savefig(os.path.join(output_dir, 'saturation_evolution.png'))
    plt.close()

    print(f"All output files have been saved in: {output_dir}")

if __name__ == "__main__":
    main()

Running time step 0
Saved: /content/lbm_output/density_plot_00000.png
Saved: /content/lbm_output/velocity_magnitude_00000.png
Saved: /content/lbm_output/velocity_streamlines_00000.png
Running time step 100
Saved: /content/lbm_output/density_plot_00100.png
Saved: /content/lbm_output/velocity_magnitude_00100.png
Saved: /content/lbm_output/velocity_streamlines_00100.png
Running time step 200
Saved: /content/lbm_output/density_plot_00200.png
Saved: /content/lbm_output/velocity_magnitude_00200.png
Saved: /content/lbm_output/velocity_streamlines_00200.png
Running time step 300
Saved: /content/lbm_output/density_plot_00300.png
Saved: /content/lbm_output/velocity_magnitude_00300.png
Saved: /content/lbm_output/velocity_streamlines_00300.png
Running time step 400
Saved: /content/lbm_output/density_plot_00400.png
Saved: /content/lbm_output/velocity_magnitude_00400.png
Saved: /content/lbm_output/velocity_streamlines_00400.png
Running time step 500
Saved: /content/lbm_output/density_plot_00500.png



Phase Separation Table:
   ρi    S_th    ρ_liq^cal    ρ_gas^cal    S_cal
-----  ------  -----------  -----------  -------
1.476   0.809        1.825        0.000    1.000
1.527   0.682        2.267       -0.059    0.884
1.585   0.695        2.309       -0.065    0.884
1.645   0.700        2.380       -0.070    0.903
1.704   0.704        2.451       -0.075    0.903
1.760   0.708        2.518       -0.080    0.903
All output files have been saved in: /content/lbm_output
